<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/*irrigation_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Identifying Pivot Irrigated Land
By: Carli Beisel

Adapted from David Ketchum (Github: dgketchum)

Created on May 13, 2024

Modified on May 17, 2024

Purpose:

1) Finds pivot irrigated areas in irrigation shapefile. This code assumes that circles/arcs in the landscape represent pivot irrigation. This script finds arcs in polygon geometries of the shapefile, and writes a new attribute called "pivot". This Script is directly from David Ketchum.

2) Sums area of pivot irrigation for each shapefile and combines into one dataframe. The created dataframe that has two columns: year and total acres of pivot irrigation.

3) Generates a regression plot of irrigation change over time with the genderated dataframe.

## 1. Find pivot irrigated areas in irrigation shapefile.

This code assumes that circles/arcs in the landscape represent pivot irrigation. This script finds arcs in polygon geometries of the shapefile, and writes a new attribute called "pivot".

This script is directly from David Ketchum.

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
! pip install pandarallel
from pandarallel import pandarallel
import glob
import os
from shapely.geometry import shape

#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Function to crop irrigated field shapefiles to drains

drainsheds = gpd.read_file('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_Merge_07072022.shp')
names = drainsheds['Name']
irrigated_field_files = glob.glob('/content/drive/MyDrive/Data/irrigation_shapefiles/irrigated_land_only/*.shp')
output_dir = '/content/drive/MyDrive/Data/Model Modifications/irrigation_change/drainshed_irrigated_fields'

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Create function to crop shapefile to each drainshed area
def crop_shapefiles(irrigated_field_files, drainsheds, output_dir):
    mask_gdf = drainsheds
    for field_file in irrigated_field_files:
        input_gdf = gpd.read_file(field_file)
        if input_gdf.crs != mask_gdf.crs:
            input_gdf = input_gdf.to_crs(mask_gdf.crs)
        for i, mask_feature in mask_gdf.iterrows():
            mask_geom = mask_feature['geometry']
            mask_name = mask_feature['Name']
            cropped_gdf = gpd.overlay(input_gdf, gpd.GeoDataFrame(geometry=[mask_geom], crs=mask_gdf.crs), how='intersection')
            field_name = os.path.splitext(os.path.basename(field_file))[0]
            output_filename = os.path.join(output_dir, f"{field_name}_{mask_name}_cropped.shp")
            cropped_gdf.to_file(output_filename)
            print(f"Cropped shapefile saved to {output_filename}")

crop_shapefiles(irrigated_field_files, drainsheds, output_dir)

In [ ]:
# Function to find arcs within a shapefile

min_arc = 10
tol = 0.22

def area_flood_irrigation(shp):
    df = gpd.read_file(shp)
    p = df[df['IType'] == 'P']['geometry']
    p = np.sum([g.area for g in p])
    s = df[df['IType'] == 'S']['geometry']
    s = np.sum([g.area for g in s])
    f = df[df['IType'] == 'F']['geometry']
    f = np.sum([g.area for g in f])
    t = p + s + f
    print('pivot: {:.3f} sqkm, {:.3f}'.format(p / 1e6, p / t))
    print('sprinkler: {:.3f} sqkm, {:.3f}'.format(s / 1e6, s / t))
    print('flood: {:.3f} sqkm, {:.3f}'.format(f / 1e6, f / t))

def bearing(a, b):
    lat1 = np.radians(a[0])
    lat2 = np.radians(b[0])

    diffLong = np.radians(b[1] - a[1])

    x = np.sin(diffLong) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1)
                                       * np.cos(lat2) * np.cos(diffLong))

    return np.arctan2(x, y)

def find_arcs(g):
    verts = g.exterior.coords
    arc_ct, b_prev = 0, np.pi
    for i, v in enumerate(verts):
        try:
            next = verts[i + 1]
        except IndexError:
            break
        b = bearing(v, next)
        diff = b - b_prev
        if diff < tol:
            arc_ct += 1
            if arc_ct >= min_arc:
                return True
        else:
            arc_ct = 0
        b_prev = b

    return False

def pivot_test(in_shp, out_shp):
    pandarallel.initialize(use_memory_fs=False, progress_bar=True)

    df = gpd.read_file(in_shp).explode()
    df.index = range(df.shape[0])
    print('{} features'.format(df.shape[0]))
    df['arc'] = df.geometry.apply(lambda g: find_arcs(g))
    df['arc'] = df.geometry.parallel_apply(find_arcs)
    df.to_file(out_shp, crs='epsg:4326')
    print('{} of {} features have an arc'.format(np.count_nonzero(df['arc']), df.shape[0]))

In [ ]:
#input file paths for irrigaton shapefiles
years = np.arange(1987,2022)
files = glob.glob('/content/drive/MyDrive/Data/Model Modifications/irrigation_change/drainshed_irrigated_fields/*.shp')

if __name__ == '__main__':
    for file in files:
       base_name = file.split('/')[-1].split('.')[0].replace('irrigated_', '')
       base_name = base_name.replace('_cropped', '')  # Remove "_cropped"
       for year in years:
            out_shp = f'/content/drive/MyDrive/Data/Model Modifications/irrigation_change/drainshed_pivot/{base_name}_arcs.shp'
            pivot_test(file, out_shp)

## 2. Sums area of pivot irrigated land and total irrigated land for each shapefile and combines into one dataframe.

The created dataframe that has three columns: 1) year, 2) total pivot irrigated acres, and 3) total acres irrigated.

In [ ]:
#import libraries
!pip install geopandas shapely
import geopandas as gpd
import glob
import pandas as pd
import re
from shapely.geometry import MultiPolygon #for shapefiles with holes or difficult shapes

#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the total irrigated land shapefiles
irrigated_fields = '/content/drive/MyDrive/Data/irrigation_shapefiles/irrigated_land_only/*.shp'
shapefiles = glob.glob(irrigated_fields)

# Define a function to extract the year from the filename
def extract_year(filename):
    match = re.search(r'(\d{4})', filename)
    return int(match.group(1)) if match else None

# Iterate over each shapefile and read the "Acres" column
irrigated_acres = []
for shp in shapefiles:
    year = extract_year(shp)
    if year is None:
        continue

    gdf = gpd.read_file(shp)
    total_area_acres = gdf['Acres'].sum()
    irrigated_acres.append({'Year': year, 'Total Irrigated Acres': total_area_acres})

# Create and sort the DataFrame by Year
all = pd.DataFrame(irrigated_acres).sort_values(by='Year').reset_index(drop=True)

print(all)

In [ ]:
# Load the pivot irrigation shapefiles
pivot_files = '/content/drive/MyDrive/Data/irrigation_shapefiles/pivot/*.shp'
shapefiles = glob.glob(pivot_files)

# Define a function to extract the year from the filename
def extract_year(filename):
    match = re.search(r'(\d{4})', filename)
    return int(match.group(1)) if match else None

# Iterate over each shapefile and read the "Acres" column where "arc" > 0
pivot_acres = []
for shp in shapefiles:
    year = extract_year(shp)
    if year is None:
        continue

    gdf = gpd.read_file(shp)
    # Filter rows where 'arc' column has values greater than 0
    filtered_gdf = gdf[gdf['arc'] > 0]
    # Sum the 'Acres' column for the filtered rows
    total_area_acres = filtered_gdf['Acres'].sum()
    pivot_acres.append({'Year': year, 'Pivot Acres': total_area_acres})

# Create and sort the DataFrame by Year
pivot = pd.DataFrame(pivot_acres).sort_values(by='Year').reset_index(drop=True)

print(pivot)

In [ ]:
#merge pivot and all irrigation style dataframes
merged_data = pd.merge(all, pivot, on='Year', how='inner')
print(merged_data)
merged_data.to_csv('/content/drive/MyDrive/Data/irrigation_shapefiles/irrigation_change.csv', index=False)

## 3. Generate a regression plot of irrigation change over time for each drainshed.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/irrigation_shapefiles/irrigation_change.csv')

In [ ]:
# Calculate proportion of pivot irrigated acres divided by total irrigated acres
df['Proportion_Pivot'] = df['Pivot Acres'] / df['Total Irrigated Acres'] #old version
print(df)

In [ ]:
# Initialize and fit the linear regression model
model = LinearRegression()
X = df[['Year']]
y = df['Proportion_Pivot']
model.fit(X, y)

# Predict values using the fitted model
df['Predicted_Acres'] = model.predict(X)

# Calculate the R² value
r_squared = model.score(X, y)

# Plot scatter plot and regression line
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Year', y='Proportion_Pivot', marker='o', label='Actual Data')
plt.plot(df['Year'], df['Predicted_Acres'], color='red', label='Regression Line')

# Customize the plot
plt.title('Proportion of Land Pivot Irrigated each Year')
plt.xlabel('Year')
plt.ylabel('Proportion of Land Irrigated with Pivot')
plt.grid(True)
plt.legend()

# Annotate the R² value on the plot
plt.text(
    0.05, 0.85, f'R²: {r_squared:.2f}',
    transform=plt.gca().transAxes, fontsize=12, verticalalignment='top'
)

# Show the plot
plt.savefig('/content/drive/MyDrive/Carli Thesis/Figures/pivot_regression.tiff', format='tiff', dpi=300)
plt.show()

In [ ]:
# Line graph with pivot proportion
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Year', y='Proportion_Pivot', marker='o', label='Actual Data')
plt.plot(df['Year'], df['Proportion_Pivot'], color='red', label='Regression Line')

#Customize the legend
plt.title('Proportion of Land Irrigated with Pivot')
plt.xlabel('Year')
plt.ylabel('Proportion of Land Irrigated with Pivot')
plt.grid(True)
plt.legend()

plt.savefig('/content/drive/MyDrive/Carli Thesis/Figures/pivot_scatter.tiff', format='tiff', dpi=300)

## 4. Create a new dataset based on regression data for input into GLMM model.